In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

# Feature Extraction con BERT

In questo notebook è stato usato BERT, senza fine-tuning, per estrarre le feature dai testi, quindi è stato usato senza la testa di classificazione. Questo passaggio è servito per il pre-addestramento della testa di classificazione personalizzata.

In [ ]:
import os
import numpy as np
import pandas as pd
import datasets
import evaluate
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModel
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
import torch

In [ ]:
# nome modello
model_name = 'dbmdz/bert-base-italian-cased'

# carico il modello
model = AutoModel.from_pretrained(model_name)

# tokenizzatore associato
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
# carico i dataset tokenizzati
train = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_train')
val = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_val')
test = datasets.load_from_disk('/content/drive/MyDrive/Colab Notebooks/hf_tokenized_test')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class FeatureExtractor(nn.Module):
  """
  Modello usato solo per estrarre le feature da bert.
  """
  def __init__(self):
    super().__init__()
    self.bert = model


  def forward(self, input_ids, attention_mask):
      # faccio manualmente il padding
      pad = torch.zeros(input_ids.shape[0], 512 - input_ids.shape[1], device=torch.device('cuda')).long()
      input_ids = torch.cat((input_ids, pad), axis=1)
      attention_mask = torch.cat((attention_mask, pad), axis=1)
      x = self.bert(input_ids, attention_mask).pooler_output

      return x


In [ ]:
feature_ex = FeatureExtractor()
feature_ex.to('cuda')

FeatureExtractor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_model_output(model, data_in, n_sample):
  """
  Funzione per estrarre le feature dai dati, in output una matrice numpy (n, 768) e l'array delle label
  """
  data_out = np.zeros((n_sample, 768))
  y_out = np.zeros(n_sample)
  i = 0
  while i < data_out.shape[0]:
    # faccio l'estrazione in batch di 10 istanze
    input_ids = data_in[i:i+10]['input_ids']
    attention_mask = data_in[i:i+10]['attention_mask']
    labels = data_in[i:i+10]['label']
    # sposto su GPU
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    with torch.no_grad():
      model_out = model(input_ids, attention_mask)

    # riporto sulla cpu e riempio la matrice di output
    model_out.to('cpu')
    data_out[i:i+10, :] = model_out.cpu()
    y_out[i:i+10] = labels
    i+=10

    if i % 100 == 0:
      print(i)

  return data_out, y_out

In [ ]:
X_val, y_val = get_model_output(feature_ex, val, val.shape[0])
X_train, y_train = get_model_output(feature_ex, train, train.shape[0])
X_test, y_test = get_model_output(feature_ex, test, test.shape[0])

100
200
300
400
500
600
700
800
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
100
200
300
400
500
600
700
800


In [ ]:
X_train.shape, X_val.shape, X_test.shape

((7200, 768), (800, 768), (800, 768))

In [ ]:
# salvo i dataset
np.savez('/content/drive/My Drive/feat_ex_data', X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)